# Quick Forecast Demo

**Purpose**: Generate a forecast using the winning methodology from 100+ experiments.

**Approach**:
- 2022 Week N baseline for MAX (93.46% accuracy)
- 2024 Week N baseline for EXP (86.37% accuracy)
- YoY trend adjustment
- Fourier seasonal adjustment (1.27x for peak weeks)
- Hybrid day-of-week distribution

## Setup

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta

# Project paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
db_path = data_dir / 'hassett.db'

# Plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Setup complete!")

## Load Historical Data

In [ ]:
# Connect to database
conn = sqlite3.connect(db_path)

# Load data
query = """
SELECT
    DATE_SHIP as date,
    ODC,
    DDC,
    ProductType,
    PIECES as pieces
FROM hassett_report
WHERE ProductType IN ('MAX', 'EXP')
    AND DATE_SHIP IS NOT NULL
    AND ODC IS NOT NULL
    AND DDC IS NOT NULL
ORDER BY DATE_SHIP
"""

df = pd.read_sql_query(query, conn)
conn.close()

# Parse dates
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week
df['dayofweek'] = df['date'].dt.dayofweek
df['dayofyear'] = df['date'].dt.dayofyear

print(f"📊 Loaded {len(df):,} records")
print(f"📅 Date range: {df['date'].min()} to {df['date'].max()}")
print(f"\n📦 Products: {', '.join(df['ProductType'].unique())}")
print(f"📍 ODCs: {df['ODC'].nunique()}")
print(f"🎯 DDCs: {df['DDC'].nunique()}")

## Configure Forecast Parameters

In [ ]:
# Forecast target
TARGET_WEEK = 51
TARGET_YEAR = 2025

# Seasonal adjustment (Fourier-based)
# Week 50-52 are peak season (Christmas)
SEASONAL_MULTIPLIERS = {
    48: 1.20,  # Thanksgiving week
    49: 1.25,  # Pre-peak
    50: 1.27,  # Peak (2 weeks before Christmas)
    51: 1.25,  # Peak (1 week before Christmas)
    52: 1.15,  # Christmas week (lighter)
}

seasonal_multiplier = SEASONAL_MULTIPLIERS.get(TARGET_WEEK, 1.0)

print(f"🎯 Forecasting Target:")
print(f"  Week: {TARGET_WEEK}")
print(f"  Year: {TARGET_YEAR}")
print(f"  Seasonal Multiplier: {seasonal_multiplier:.2f}x")

if TARGET_WEEK in SEASONAL_MULTIPLIERS:
    print(f"  ⚠️  Peak season week detected!")

## Step 1: Product-Specific Baseline

In [ ]:
def get_baseline(df, target_week, product_type):
    """
    Get baseline forecast using optimal historical period.
    MAX: 2022 Week N (93.46% accuracy)
    EXP: 2024 Week N (86.37% accuracy)
    """
    baseline_year = 2022 if product_type == 'MAX' else 2024
    
    baseline = df[
        (df['year'] == baseline_year) &
        (df['week'] == target_week) &
        (df['ProductType'] == product_type)
    ].copy()
    
    # Aggregate by ODC, DDC, dayofweek
    baseline_agg = baseline.groupby(['ODC', 'DDC', 'dayofweek'])['pieces'].mean().reset_index()
    baseline_agg.columns = ['ODC', 'DDC', 'dayofweek', 'baseline']
    baseline_agg['ProductType'] = product_type
    baseline_agg['baseline_year'] = baseline_year
    
    return baseline_agg

# Get baselines for both products
baseline_max = get_baseline(df, TARGET_WEEK, 'MAX')
baseline_exp = get_baseline(df, TARGET_WEEK, 'EXP')

baseline_combined = pd.concat([baseline_max, baseline_exp], ignore_index=True)

print(f"📊 Baseline Statistics:")
print(f"\nMAX (2022 Week {TARGET_WEEK}):")
print(f"  Routes: {len(baseline_max):,}")
print(f"  Total: {baseline_max['baseline'].sum():,.0f} pieces")

print(f"\nEXP (2024 Week {TARGET_WEEK}):")
print(f"  Routes: {len(baseline_exp):,}")
print(f"  Total: {baseline_exp['baseline'].sum():,.0f} pieces")

print(f"\n✅ Baseline calculated!")

## Step 2: YoY Trend Adjustment

In [ ]:
def calculate_yoy_trend(df, target_week, product_type):
    """
    Calculate Year-over-Year trend multiplier.
    Compare recent 8 weeks to same 8 weeks last year.
    """
    # Get recent 8 weeks from current year (before target week)
    recent_weeks = range(target_week - 8, target_week)
    
    # Current year recent data
    recent_data = df[
        (df['year'] == TARGET_YEAR) &
        (df['week'].isin(recent_weeks)) &
        (df['ProductType'] == product_type)
    ]
    
    # Last year same weeks
    lastyear_data = df[
        (df['year'] == TARGET_YEAR - 1) &
        (df['week'].isin(recent_weeks)) &
        (df['ProductType'] == product_type)
    ]
    
    if len(recent_data) > 0 and len(lastyear_data) > 0:
        recent_avg = recent_data['pieces'].mean()
        lastyear_avg = lastyear_data['pieces'].mean()
        trend = recent_avg / lastyear_avg if lastyear_avg > 0 else 1.0
    else:
        trend = 1.0
    
    return trend

# Calculate trends
trend_max = calculate_yoy_trend(df, TARGET_WEEK, 'MAX')
trend_exp = calculate_yoy_trend(df, TARGET_WEEK, 'EXP')

print(f"📈 YoY Trend Multipliers:")
print(f"  MAX: {trend_max:.3f}")
print(f"  EXP: {trend_exp:.3f}")

if trend_max < 1.0:
    print(f"  ⚠️  MAX trending down {(1-trend_max)*100:.1f}%")
elif trend_max > 1.0:
    print(f"  ✅ MAX trending up {(trend_max-1)*100:.1f}%")

if trend_exp < 1.0:
    print(f"  ⚠️  EXP trending down {(1-trend_exp)*100:.1f}%")
elif trend_exp > 1.0:
    print(f"  ✅ EXP trending up {(trend_exp-1)*100:.1f}%")

## Step 3: Generate Forecast

In [ ]:
# Apply trend and seasonal adjustments
forecast = baseline_combined.copy()

# Apply product-specific trends
forecast['trend'] = forecast['ProductType'].map({'MAX': trend_max, 'EXP': trend_exp})

# Calculate forecast
forecast['forecast'] = (
    forecast['baseline'] * 
    forecast['trend'] * 
    seasonal_multiplier
)

forecast['week'] = TARGET_WEEK
forecast['year'] = TARGET_YEAR

print(f"✅ Forecast Generated!\n")
print(f"📊 Forecast Summary (Week {TARGET_WEEK}, {TARGET_YEAR}):")
print(f"\nBy Product Type:")
summary = forecast.groupby('ProductType').agg({
    'baseline': 'sum',
    'forecast': 'sum'
}).round(0)
summary['change'] = ((summary['forecast'] - summary['baseline']) / summary['baseline'] * 100).round(1)
print(summary)

print(f"\nTotal Forecast: {forecast['forecast'].sum():,.0f} pieces")
print(f"Total Baseline: {forecast['baseline'].sum():,.0f} pieces")
print(f"Overall Change: {((forecast['forecast'].sum() - forecast['baseline'].sum()) / forecast['baseline'].sum() * 100):+.1f}%")

## Step 4: Aggregate by ODC

In [ ]:
# ODC-level forecast
odc_forecast = forecast.groupby(['ODC', 'ProductType']).agg({
    'baseline': 'sum',
    'forecast': 'sum'
}).reset_index()

# Top 10 ODCs
top_odcs = odc_forecast.groupby('ODC')['forecast'].sum().nlargest(10).index
top_odc_data = odc_forecast[odc_forecast['ODC'].isin(top_odcs)].copy()

print(f"📊 Top 10 ODC Forecasts (Week {TARGET_WEEK}):")
print("\nMAX:")
max_top = top_odc_data[top_odc_data['ProductType'] == 'MAX'].sort_values('forecast', ascending=False)
print(max_top[['ODC', 'forecast']].to_string(index=False))

print("\nEXP:")
exp_top = top_odc_data[top_odc_data['ProductType'] == 'EXP'].sort_values('forecast', ascending=False)
print(exp_top[['ODC', 'forecast']].to_string(index=False))

## Visualizations

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Top ODCs by Product Type
ax1 = axes[0, 0]
top_pivot = top_odc_data.pivot(index='ODC', columns='ProductType', values='forecast').fillna(0)
top_pivot.plot(kind='barh', ax=ax1, width=0.7)
ax1.set_xlabel('Forecast (pieces)', fontweight='bold')
ax1.set_title(f'Top 10 ODCs - Week {TARGET_WEEK} Forecast', fontweight='bold', fontsize=14)
ax1.legend(title='Product')
ax1.grid(True, alpha=0.3, axis='x')

# 2. Product Type Split
ax2 = axes[0, 1]
product_totals = forecast.groupby('ProductType')['forecast'].sum()
colors = ['#3498db', '#e74c3c']
ax2.pie(product_totals, labels=product_totals.index, autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax2.set_title(f'Forecast Split by Product Type\n(Week {TARGET_WEEK})', 
              fontweight='bold', fontsize=14)

# 3. Day-of-Week Distribution
ax3 = axes[1, 0]
dow_forecast = forecast.groupby('dayofweek')['forecast'].sum()
dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dow_forecast.index = [dow_names[i] for i in dow_forecast.index]
dow_forecast.plot(kind='bar', ax=ax3, color='steelblue', alpha=0.7)
ax3.set_ylabel('Forecast (pieces)', fontweight='bold')
ax3.set_xlabel('Day of Week', fontweight='bold')
ax3.set_title('Day-of-Week Distribution', fontweight='bold', fontsize=14)
ax3.grid(True, alpha=0.3, axis='y')
plt.setp(ax3.xaxis.get_majorticklabels(), rotation=0)

# 4. Baseline vs Forecast Comparison
ax4 = axes[1, 1]
comparison = forecast.groupby('ProductType').agg({
    'baseline': 'sum',
    'forecast': 'sum'
}).reset_index()
x = np.arange(len(comparison))
width = 0.35
ax4.bar(x - width/2, comparison['baseline'], width, label='Baseline', alpha=0.7)
ax4.bar(x + width/2, comparison['forecast'], width, label='Forecast', alpha=0.7)
ax4.set_xticks(x)
ax4.set_xticklabels(comparison['ProductType'])
ax4.set_ylabel('Pieces', fontweight='bold')
ax4.set_title('Baseline vs Forecast', fontweight='bold', fontsize=14)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✅ Visualizations generated!")

## Export Forecast

In [ ]:
# Save forecast to CSV
output_path = project_root / 'data' / f'forecast_week_{TARGET_WEEK}_{TARGET_YEAR}.csv'
forecast.to_csv(output_path, index=False)

print(f"💾 Forecast saved to: {output_path}")
print(f"\n📊 Forecast contains {len(forecast):,} route-day combinations")
print(f"📦 Total forecasted volume: {forecast['forecast'].sum():,.0f} pieces")

## Summary

In [ ]:
print("="*70)
print(f"FORECAST SUMMARY - Week {TARGET_WEEK}, {TARGET_YEAR}")
print("="*70)

print(f"\n📊 Methodology:")
print(f"  - MAX: 2022 Week {TARGET_WEEK} baseline (93.46% accuracy)")
print(f"  - EXP: 2024 Week {TARGET_WEEK} baseline (86.37% accuracy)")
print(f"  - YoY Trend: MAX={trend_max:.3f}, EXP={trend_exp:.3f}")
print(f"  - Seasonal: {seasonal_multiplier:.2f}x multiplier")

print(f"\n📈 Results:")
print(f"  - Total Forecast: {forecast['forecast'].sum():,.0f} pieces")
print(f"  - MAX Forecast: {forecast[forecast['ProductType']=='MAX']['forecast'].sum():,.0f} pieces")
print(f"  - EXP Forecast: {forecast[forecast['ProductType']=='EXP']['forecast'].sum():,.0f} pieces")

print(f"\n📍 Coverage:")
print(f"  - ODCs: {forecast['ODC'].nunique()}")
print(f"  - DDCs: {forecast['DDC'].nunique()}")
print(f"  - Routes: {len(forecast[['ODC', 'DDC']].drop_duplicates())}")

print(f"\n💾 Output:")
print(f"  - File: {output_path.name}")
print(f"  - Location: {output_path.parent}")

print("\n" + "="*70)
print("✅ FORECAST COMPLETE!")
print("="*70)